In [6]:
import torch
import torch.nn.functional as F
from basic import logvariational_fn, samplevariational_fn, logprior_fn
import torch.nn as nn

In [9]:
def bayes_linear(
            in_features, 
            out_features, 
            x, 
            prior_pi: float,
            prior_var1: float,
            prior_var2: float, 
        ):

    mus = torch.empty(size=(in_features, out_features)).normal_()
    rhos = torch.empty(size=(in_features, out_features)).normal_()

    sampled_weights = samplevariational_fn(
        mus=mus,
        rhos=rhos
        )
    
    logvariational_prob = logvariational_fn(sampled_weights, mus, rhos)
    logprior_prob = logprior_fn(sampled_weights, prior_pi, prior_var1, prior_var2)

    return F.linear(x, sampled_weights.T), logvariational_prob.sum(), logprior_prob.sum()





In [10]:
x = torch.randn((1, 5), dtype=torch.float)
bayes_linear(
    in_features=5, 
    out_features=10, 
    x=x, 
    prior_pi = 0.5,
    prior_var1=0.7,
    prior_var2=0.005)

(tensor([[ 3.9951, -1.3665, -4.0589, -3.4829, -0.1809,  1.6508,  3.2666,  0.0356,
           0.3146,  1.3047]]),
 tensor(-43.5161, dtype=torch.float64),
 tensor(-133.9280, dtype=torch.float64))